In [1]:
import os, re
import glob
import cv2
import numpy as np
import cupy as cp
from cupyx.scipy import ndimage as cpx_ndimage  # Import CuPy's GPU ndimage module
import tifffile as tiff
import scipy as sp
from scipy import ndimage, io as sio
from scipy.ndimage import maximum_filter, label, find_objects
from scipy.stats import chi2
from skimage.feature import peak_local_max
from natsort import natsorted
import trackpy as tp
import pandas as pd
import multiprocessing as mp
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.widgets import Slider
from matplotlib import animation, rc
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display, Image

# Set up matplotlib for animations
rc('animation', html='jshtml')

# Suppress specific warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)  # ignore warnings for specific matplotlib commands which will be outdated soon


In [128]:
data_folder1 = convert_to_code_path(r'E:\Spandan\2D_Neurons_Paper\Glass\div2\control_div2_glass_2019_06_24_cortical_div2_timelapse2_1');
data_folder2 = convert_to_code_path(r'E:\Spandan\2D_Neurons_Paper\Glass\div2\control_div2_glass_2019_06_24_cortical_div2_timelapse1');

control_div2_glass_2019_06_24_cortical_div2_timelapse2_1 = pd.read_csv(data_folder1+'\\tp_trajectories_final.csv')
control_div2_glass_2019_06_24_cortical_div2_timelapse1 = pd.read_csv(data_folder2+'\\tp_trajectories_final.csv')

# Merge along a new axis using MultiIndex
merged_df = pd.concat({'df1': df1, 'df2': df2}, axis=0)

# print(merged_df)

# Save DataFrame as CSV
merged_df.to_csv(data'test.csv', index=True)

In [13]:
def convert_to_code_path(windows_path):
    """
    Converts a Windows file path with single backslashes to a format with double backslashes for use in Python code.

    Parameters:
    - windows_path: str
        The original Windows file path (e.g., 'C:\\Users\\YourName\\Folder').

    Returns:
    - str
        The modified path with double backslashes, suitable for Python (e.g., 'C:\\\\Users\\\\YourName\\\\Folder').
    """
    # Replace each single backslash with a double backslash
    return windows_path.replace("\\", "\\\\")

def extract_numeric(value):
    """
    Extracts numeric value from a given input. If the input is not numeric,
    it will return None.

    Args:
    - value: The input value to be processed.

    Returns:
    - float or None: The extracted numeric value, or None if the input is not valid.
    """
    # Convert to string and strip any leading/trailing whitespace
    value_str = str(value).strip()

    # Use regex to find numeric values (including decimals)
    match = re.findall(r"[-+]?\d*\.\d+|\d+", value_str)

    if match:
        return float(match[0])  # Return the first found numeric value
    else:
        return None


def get_file_names_and_params_by_div(sheet_path, div_value, sheet_name='ridges'):
    """
    Get all 'file name', 'resolution', and 'frame interval' values from the specified Excel sheet where div equals the given value.

    Args:
    - sheet_path: Path to the Excel sheet containing imaging details.
    - div_value: The 'div' value to filter the Excel sheet.
    - sheet_name: The sheet name in the Excel file (default: 'ridges').

    Returns:
    - params: A list of tuples containing ('file name', 'resolution', 'frame interval') where div equals div_value.
    """
    # Load the Excel sheet
    df_sheet = pd.read_excel(sheet_path, sheet_name=sheet_name)

    # Filter based on the 'div' value
    df_sheet_filtered = df_sheet[df_sheet['div'] == div_value]

    # Get the parameters
    params = []
    for index in df_sheet_filtered.index:
        # Extract file name
        file_name = df_sheet_filtered['file name'][index]

        # Extract frame interval
        interval = df_sheet_filtered['frame interval'][index]
        integer_interval_value = extract_numeric(interval)

        # Extract resolution
        resolution = df_sheet_filtered['resolution'][index]
        integer_resolution_value = extract_numeric(resolution)

        # Append to params
        params.append((file_name, integer_resolution_value, integer_interval_value))

    return params

In [5]:
def check_level_2_subfolders_by_div_and_params(main_folder, sheet_path, div_value, resolution_range, frame_interval_range):
    """
    Check all level-2 subfolder names under level-1 subfolders that match the specified div value,
    and verify if their corresponding resolution and frame interval match the criteria.
    This function combines data from 'tp_trajectories_final.csv' files and 'fit_' files into one large DataFrame.
    
    Args:
    - main_folder: Path to the main folder containing level-1 subfolders.
    - sheet_path: Path to the Excel sheet containing imaging details.
    - div_value: The 'div' value to filter the Excel sheet.
    - resolution_range: The resolution range to match (as a tuple or list).
    - frame_interval_range: The frame interval range to match (as a tuple or list).
    """
    
    # Get the file names and corresponding parameters that match the div value
    params = get_file_names_and_params_by_div(sheet_path, div_value)
    if not params:
        print(f"No parameters found for div = {div_value}.")
        return

    # Create an empty DataFrame to store combined data
    # combined_df = pd.DataFrame()

    # Loop through each level-1 subfolder
    for subfolder in os.listdir(main_folder):
        subfolder_path = os.path.join(main_folder, subfolder)

        # Check if it's a directory (level-1)
        if os.path.isdir(subfolder_path):
            # Loop through each level-2 subfolder
            for level_2_subfolder in os.listdir(subfolder_path):
                level_2_subfolder_path = os.path.join(subfolder_path, level_2_subfolder)

                if os.path.isdir(level_2_subfolder_path):
                    # Check if this level-2 folder corresponds to any file name
                    for file_name, resolution, frame_interval in params:
                        if file_name in level_2_subfolder:
                            print(f"Analyzing Level-2 Subfolder: {level_2_subfolder}")

                            # Verify if resolution and frame interval meet the criteria
                            if ((resolution_range is None or (resolution is not None and resolution_range[0] <= resolution <= resolution_range[1])) and
                                (frame_interval_range is None or (frame_interval is not None and frame_interval_range[0] <= frame_interval <= frame_interval_range[1]))):
                                
                                print("Criteria Met")

                                # Find 'tp_trajectories_final.csv' and 'fit_' files
                                try:
                                    tp_trajectories_path = convert_to_code_path(os.path.join(level_2_subfolder_path, 'tp_trajectories_final.csv'))
                                    
                                    if os.path.exists(tp_trajectories_path):
                                        # Load tp_trajectories_final.csv
                                        tp_trajectories_df = pd.read_csv(tp_trajectories_path, encoding='ISO-8859-1')
                                        num_unique_tracks = tp_trajectories_df['particle'].nunique()
                                        print(num_unique_tracks)
                                        
                                        # # Add context columns at the start
                                        # tp_trajectories_df.insert(0, 'div_stage', subfolder)  # Add level-1 folder name
                                        # tp_trajectories_df.insert(1, 'file_name', level_2_subfolder)  # Add level-2 folder name
                                        
                                        # # Find and add 'fit_' CSV files as new columns
                                        # for file in os.listdir(level_2_subfolder_path):
                                        #     if file.startswith('fit_') and file.endswith('.csv'):
                                        #         fit_file_path = convert_to_code_path(os.path.join(level_2_subfolder_path, file))
                                        #         fit_df = pd.read_csv(fit_file_path, encoding='ISO-8859-1')
                                        #         # Use the 'fit_' filename as the column name prefix
                                        #         fit_prefix = file.replace('.csv', '')
                                        #         fit_df = fit_df.add_prefix(f"{fit_prefix}_")
                                        #         tp_trajectories_df = pd.concat([tp_trajectories_df, fit_df], axis=1)

                                        # # Append the DataFrame to the combined DataFrame
                                        # combined_df = pd.concat([combined_df, tp_trajectories_df], ignore_index=True)
                                    else:
                                        print(f"No 'tp_trajectories_final.csv' file found in {level_2_subfolder_path}.")
                                
                                except Exception as e:
                                    print(f"Error analyzing data in {level_2_subfolder}: {e}")
                            else:
                                print("Criteria Not Met")

    # # Save the combined DataFrame to a CSV file at the end
    # if not combined_df.empty:
    #     combined_df.to_csv(os.path.join(main_folder, 'combined_trajectories_final.csv'), index=False)
    #     print("Combined .csv file saved as 'combined_trajectories_final.csv'.")
    # else:
    #     print("No valid data to combine.")


In [14]:
def analyze_all_divs(main_folder, sheet_path, resolution_range, frame_interval_range):
    """
    Analyze all level-1 folders corresponding to different DIV values.

    Parameters:
    - main_folder: Path to the main folder containing level-1 subfolders.
    - sheet_path: Path to the Excel file with additional data.
    - resolution_range: Tuple containing the range of resolutions to filter.
    - frame_interval_range: Tuple containing the range of frame intervals to filter.
    """
    
    # Loop through each level-1 subfolder
    for subfolder in os.listdir(main_folder):
        subfolder_path = os.path.join(main_folder, subfolder)

        # Check if it's a directory (level-1)
        if os.path.isdir(subfolder_path):
            # Use regex to extract the DIV value from the folder name
            match = re.search(r'div(\d+)', subfolder, re.IGNORECASE)
            if match:
                div_value = int(match.group(1))  # Convert extracted string to integer
                print(f"Analyzing {subfolder} with DIV value: {div_value}")

                # Call your analysis function
                check_level_2_subfolders_by_div_and_params(main_folder, sheet_path, div_value, resolution_range, frame_interval_range)

# Example usage

main_folder = r'E:\Spandan\2D_Neurons_Paper\Ridges'
sheet_path = r'E:\Spandan\Kate\NEURON MOVIES\tifNotes.xlsx'
resolution_range = (2.6, 2.9)
frame_interval_range = (1.8, 2.2)

analyze_all_divs(main_folder, sheet_path, resolution_range, frame_interval_range)


Analyzing div2 with DIV value: 2
Analyzing Level-2 Subfolder: ridges_19_01_21_div2cortex_B1_timelapse2
Criteria Met
17
Analyzing Level-2 Subfolder: ridges_19_01_21_div2cortex_B1_timelapse3
Criteria Met
17
Analyzing Level-2 Subfolder: ridges_19_01_21_div2cortex_B1_timelapse5
Criteria Met
18
Analyzing Level-2 Subfolder: ridges_19_01_21_div2cortex_B1_timelapse6
Criteria Met
20
Analyzing Level-2 Subfolder: ridges_19_02_11_C2_ridges_timelapse1
Criteria Met
51
Analyzing Level-2 Subfolder: ridges_19_02_11_C2_ridges_timelapse4a
Criteria Met
28
Analyzing Level-2 Subfolder: ridges_19_02_11_C2_ridges_timelapse7
Criteria Met
10
Analyzing Level-2 Subfolder: ridges_2019_02_11_C2_ridges_timelapse1
Criteria Met
No 'tp_trajectories_final.csv' file found in E:\Spandan\2D_Neurons_Paper\Ridges\div4\ridges_2019_02_11_C2_ridges_timelapse1.
Analyzing Level-2 Subfolder: ridges_2019_02_11_C2_ridges_timelapse1
Criteria Met
53
Analyzing div3 with DIV value: 3
Analyzing Level-2 Subfolder: ridges_2021_04_16_MAX_we

In [108]:
yo = pd.read_csv('E:\\Spandan\\2D_Neurons_Paper\\Glass\\div2\\control_div2_glass_2019_01_23_div2cortex_A1_timelapse1\\tp_trajectories_final.csv')

In [3]:
def convert_to_code_path(windows_path):
    """
    Converts a Windows file path with single backslashes to a format with double backslashes for use in Python code.

    Parameters:
    - windows_path: str
        The original Windows file path (e.g., 'C:\\Users\\YourName\\Folder').

    Returns:
    - str
        The modified path with double backslashes, suitable for Python (e.g., 'C:\\\\Users\\\\YourName\\\\Folder').
    """
    # Replace each single backslash with a double backslash
    return windows_path.replace("\\", "\\\\")

def extract_numeric(value):
    """
    Extracts numeric value from a given input. If the input is not numeric,
    it will return None.

    Args:
    - value: The input value to be processed.

    Returns:
    - float or None: The extracted numeric value, or None if the input is not valid.
    """
    # Convert to string and strip any leading/trailing whitespace
    value_str = str(value).strip()

    # Use regex to find numeric values (including decimals)
    match = re.findall(r"[-+]?\d*\.\d+|\d+", value_str)

    if match:
        return float(match[0])  # Return the first found numeric value
    else:
        return None


def get_file_names_and_params_by_div(sheet_path, div_value, sheet_name='glass'):
    """
    Get all 'file name', 'resolution', and 'frame interval' values from the specified Excel sheet where div equals the given value.

    Args:
    - sheet_path: Path to the Excel sheet containing imaging details.
    - div_value: The 'div' value to filter the Excel sheet.
    - sheet_name: The sheet name in the Excel file (default: 'glass').

    Returns:
    - params: A list of tuples containing ('file name', 'resolution', 'frame interval') where div equals div_value.
    """
    # Load the Excel sheet
    df_sheet = pd.read_excel(sheet_path, sheet_name=sheet_name)

    # Filter based on the 'div' value
    df_sheet_filtered = df_sheet[df_sheet['div'] == div_value]

    # Get the parameters
    params = []
    for index in df_sheet_filtered.index:
        # Extract file name
        file_name = df_sheet_filtered['file name'][index]

        # Extract frame interval
        interval = df_sheet_filtered['frame interval'][index]
        integer_interval_value = extract_numeric(interval)

        # Extract resolution
        resolution = df_sheet_filtered['resolution'][index]
        integer_resolution_value = extract_numeric(resolution)

        # Append to params
        params.append((file_name, integer_resolution_value, integer_interval_value))

    return params

In [5]:
def check_level_2_subfolders_by_div_and_params(main_folder, sheet_path, div_value, resolution_range, frame_interval_range):
    """
    Check all level-2 subfolder names under level-1 subfolders that match the specified div value,
    and verify if their corresponding resolution and frame interval match the criteria.
    This function combines data from 'tp_trajectories_final.csv' files and 'fit_' files into one large DataFrame.
    
    Args:
    - main_folder: Path to the main folder containing level-1 subfolders.
    - sheet_path: Path to the Excel sheet containing imaging details.
    - div_value: The 'div' value to filter the Excel sheet.
    - resolution_range: The resolution range to match (as a tuple or list).
    - frame_interval_range: The frame interval range to match (as a tuple or list).
    """
    
    # Get the file names and corresponding parameters that match the div value
    params = get_file_names_and_params_by_div(sheet_path, div_value)
    if not params:
        print(f"No parameters found for div = {div_value}.")
        return

    # Create an empty DataFrame to store combined data
    # combined_df = pd.DataFrame()

    # Loop through each level-1 subfolder
    for subfolder in os.listdir(main_folder):
        subfolder_path = os.path.join(main_folder, subfolder)

        # Check if it's a directory (level-1)
        if os.path.isdir(subfolder_path):
            # Loop through each level-2 subfolder
            for level_2_subfolder in os.listdir(subfolder_path):
                level_2_subfolder_path = os.path.join(subfolder_path, level_2_subfolder)

                if os.path.isdir(level_2_subfolder_path):
                    # Check if this level-2 folder corresponds to any file name
                    for file_name, resolution, frame_interval in params:
                        if file_name in level_2_subfolder:
                            print(f"Analyzing Level-2 Subfolder: {level_2_subfolder}")

                            # Verify if resolution and frame interval meet the criteria
                            if ((resolution_range is None or (resolution is not None and resolution_range[0] <= resolution <= resolution_range[1])) and
                                (frame_interval_range is None or (frame_interval is not None and frame_interval_range[0] <= frame_interval <= frame_interval_range[1]))):
                                
                                print("Criteria Met")

                                # Find 'tp_trajectories_final.csv' and 'fit_' files
                                try:
                                    tp_trajectories_path = convert_to_code_path(os.path.join(level_2_subfolder_path, 'track_metrics.csv'))
                                    
                                    if os.path.exists(tp_trajectories_path):
                                        # Load tp_trajectories_final.csv
                                        tp_trajectories_df = pd.read_csv(tp_trajectories_path, encoding='ISO-8859-1')
                                        num_unique_tracks = tp_trajectories_df['particle'].nunique()
                                        print(num_unique_tracks)
                                        
                                        # # Add context columns at the start
                                        # tp_trajectories_df.insert(0, 'div_stage', subfolder)  # Add level-1 folder name
                                        # tp_trajectories_df.insert(1, 'file_name', level_2_subfolder)  # Add level-2 folder name
                                        
                                        # # Find and add 'fit_' CSV files as new columns
                                        # for file in os.listdir(level_2_subfolder_path):
                                        #     if file.startswith('fit_') and file.endswith('.csv'):
                                        #         fit_file_path = convert_to_code_path(os.path.join(level_2_subfolder_path, file))
                                        #         fit_df = pd.read_csv(fit_file_path, encoding='ISO-8859-1')
                                        #         # Use the 'fit_' filename as the column name prefix
                                        #         fit_prefix = file.replace('.csv', '')
                                        #         fit_df = fit_df.add_prefix(f"{fit_prefix}_")
                                        #         tp_trajectories_df = pd.concat([tp_trajectories_df, fit_df], axis=1)

                                        # # Append the DataFrame to the combined DataFrame
                                        # combined_df = pd.concat([combined_df, tp_trajectories_df], ignore_index=True)
                                    else:
                                        print(f"No 'track_metrics.csv' file found in {level_2_subfolder_path}.")
                                
                                except Exception as e:
                                    print(f"Error analyzing data in {level_2_subfolder}: {e}")
                            else:
                                print("Criteria Not Met")

    # # Save the combined DataFrame to a CSV file at the end
    # if not combined_df.empty:
    #     combined_df.to_csv(os.path.join(main_folder, 'combined_trajectories_final.csv'), index=False)
    #     print("Combined .csv file saved as 'combined_trajectories_final.csv'.")
    # else:
    #     print("No valid data to combine.")


In [7]:
def analyze_all_divs(main_folder, sheet_path, resolution_range, frame_interval_range):
    """
    Analyze all level-1 folders corresponding to different DIV values.

    Parameters:
    - main_folder: Path to the main folder containing level-1 subfolders.
    - sheet_path: Path to the Excel file with additional data.
    - resolution_range: Tuple containing the range of resolutions to filter.
    - frame_interval_range: Tuple containing the range of frame intervals to filter.
    """
    
    # Loop through each level-1 subfolder
    for subfolder in os.listdir(main_folder):
        subfolder_path = os.path.join(main_folder, subfolder)

        # Check if it's a directory (level-1)
        if os.path.isdir(subfolder_path):
            # Use regex to extract the DIV value from the folder name
            match = re.search(r'div(\d+)', subfolder, re.IGNORECASE)
            if match:
                div_value = int(match.group(1))  # Convert extracted string to integer
                print(f"Analyzing {subfolder} with DIV value: {div_value}")

                # Call your analysis function
                check_level_2_subfolders_by_div_and_params(main_folder, sheet_path, div_value, resolution_range, frame_interval_range)

# Example usage

main_folder = r'E:\Spandan\2D_Neurons_Paper\Glass'
sheet_path = r'E:\Spandan\Kate\NEURON MOVIES\tifNotes.xlsx'
resolution_range = (2.7, 2.8)
frame_interval_range = (1.9, 2.1)

analyze_all_divs(main_folder, sheet_path, resolution_range, frame_interval_range)


Analyzing div10 with DIV value: 10
Analyzing Level-2 Subfolder: control_div10_glass_2019_04_22_NewPdl_GFP RFP_5 5_488_timelapse1
Criteria Met
68
Analyzing Level-2 Subfolder: control_div10_glass_2021_04_12_MAX_6well1_div10_timelapse3_100x
Criteria Not Met
Analyzing Level-2 Subfolder: control_div10_glass_2021_12_06_dish2_control_neuron1_media
Criteria Met
109
Analyzing Level-2 Subfolder: control_div10_glass_2021_12_06_dish2_control_neuron2_media
Criteria Met
121
Analyzing Level-2 Subfolder: control_div10_glass_2021_12_06_dish2_control_neuron3_media
Criteria Met
65
Analyzing Level-2 Subfolder: control_div10_glass_2021_12_06_dish2_control_neuron4_media
Criteria Met
91
Analyzing Level-2 Subfolder: control_div10_glass_2021_12_06_dish2_control_neuron5_media
Criteria Met
82
Analyzing div11 with DIV value: 11
Analyzing Level-2 Subfolder: control_2019_02_11_CL+DIV3_Cort_timelapse2
Criteria Not Met
Analyzing Level-2 Subfolder: control_2019_02_11_CL+DIV7_Cort_timelapse5
Criteria Met
210
Analyzing 